# Tour Guide System - Results Visualization

This notebook provides comprehensive visualizations of Tour Guide experiment results.

## Contents
1. Setup and Data Loading
2. Parameter Sensitivity Charts
3. Agent Performance Analysis
4. Winner Distribution
5. Statistical Summary

In [ ]:
# Install required packages if needed
# !pip install matplotlib numpy

import json
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Set style for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("Visualization libraries loaded successfully!")

## 1. Load Experiment Data

In [ ]:
# Load results from parameter analysis
results_path = Path('../results/parameter_analysis.json')

if results_path.exists():
    with open(results_path) as f:
        data = json.load(f)
    print(f"Loaded results from {results_path}")
else:
    # Use sample data if file doesn't exist
    data = {
        'interval_experiments': [
            {'interval': 1.0, 'total_time': 12.0, 'success_rate': 0.85, 'video_wins': 3, 'music_wins': 4, 'history_wins': 3},
            {'interval': 2.0, 'total_time': 22.0, 'success_rate': 0.88, 'video_wins': 4, 'music_wins': 3, 'history_wins': 3},
            {'interval': 5.0, 'total_time': 52.0, 'success_rate': 0.95, 'video_wins': 3, 'music_wins': 3, 'history_wins': 4},
            {'interval': 10.0, 'total_time': 102.0, 'success_rate': 0.96, 'video_wins': 4, 'music_wins': 2, 'history_wins': 4},
            {'interval': 30.0, 'total_time': 302.0, 'success_rate': 0.98, 'video_wins': 5, 'music_wins': 2, 'history_wins': 3},
        ],
        'timeout_experiments': [
            {'timeout': 5.0, 'success_rate': 0.70, 'avg_score': 65},
            {'timeout': 10.0, 'success_rate': 0.85, 'avg_score': 75},
            {'timeout': 15.0, 'success_rate': 0.90, 'avg_score': 80},
            {'timeout': 30.0, 'success_rate': 0.95, 'avg_score': 85},
        ]
    }
    print("Using sample data")

print(f"Interval experiments: {len(data['interval_experiments'])}")
print(f"Timeout experiments: {len(data['timeout_experiments'])}")

## 2. Junction Interval Analysis

In [ ]:
# Extract interval data
intervals = [d['interval'] for d in data['interval_experiments']]
total_times = [d['total_time'] for d in data['interval_experiments']]
success_rates = [d['success_rate'] * 100 for d in data['interval_experiments']]

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Processing Time vs Interval
ax1.bar(range(len(intervals)), total_times, color='steelblue', alpha=0.8)
ax1.set_xticks(range(len(intervals)))
ax1.set_xticklabels([f'{i}s' for i in intervals])
ax1.set_xlabel('Junction Interval')
ax1.set_ylabel('Total Processing Time (seconds)')
ax1.set_title('Processing Time vs Junction Interval')

# Add value labels
for i, v in enumerate(total_times):
    ax1.text(i, v + 5, f'{v:.0f}s', ha='center', fontsize=10)

# Plot 2: Success Rate vs Interval
ax2.plot(intervals, success_rates, 'o-', color='green', linewidth=2, markersize=10)
ax2.fill_between(intervals, success_rates, alpha=0.3, color='green')
ax2.set_xlabel('Junction Interval (seconds)')
ax2.set_ylabel('Success Rate (%)')
ax2.set_title('Success Rate vs Junction Interval')
ax2.set_ylim([60, 102])
ax2.axhline(y=95, color='red', linestyle='--', alpha=0.5, label='95% threshold')
ax2.legend()

plt.tight_layout()
plt.savefig('../assets/interval_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Finding: Higher intervals increase reliability but take longer.")
print(f"Recommended interval: 5.0s (balances {success_rates[2]:.0f}% success with {total_times[2]:.0f}s processing)")

## 3. Agent Timeout Impact

In [ ]:
# Extract timeout data
timeouts = [d['timeout'] for d in data['timeout_experiments']]
timeout_success = [d['success_rate'] * 100 for d in data['timeout_experiments']]
avg_scores = [d['avg_score'] for d in data['timeout_experiments']]

# Create figure
fig, ax = plt.subplots(figsize=(10, 6))

# Bar positions
x = np.arange(len(timeouts))
width = 0.35

# Create bars
bars1 = ax.bar(x - width/2, timeout_success, width, label='Success Rate (%)', color='#2ecc71', alpha=0.8)
bars2 = ax.bar(x + width/2, avg_scores, width, label='Avg Quality Score', color='#3498db', alpha=0.8)

# Customize
ax.set_xlabel('Agent Timeout (seconds)')
ax.set_ylabel('Percentage / Score')
ax.set_title('Impact of Agent Timeout on Success Rate and Quality')
ax.set_xticks(x)
ax.set_xticklabels([f'{t}s' for t in timeouts])
ax.legend()
ax.set_ylim([0, 105])

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.0f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points', ha='center', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.0f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('../assets/timeout_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Finding: 30s timeout achieves optimal balance of success (95%) and quality (85).")

## 4. Winner Distribution Analysis

In [ ]:
# Aggregate winner data
total_video = sum(d['video_wins'] for d in data['interval_experiments'])
total_music = sum(d['music_wins'] for d in data['interval_experiments'])
total_history = sum(d['history_wins'] for d in data['interval_experiments'])

# Create pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
sizes = [total_video, total_music, total_history]
labels = ['Video', 'Music', 'History']
colors = ['#e74c3c', '#9b59b6', '#f39c12']
explode = (0.05, 0.05, 0.05)

ax1.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.set_title('Overall Winner Distribution')

# Stacked bar chart per interval
intervals = [str(d['interval']) + 's' for d in data['interval_experiments']]
video_wins = [d['video_wins'] for d in data['interval_experiments']]
music_wins = [d['music_wins'] for d in data['interval_experiments']]
history_wins = [d['history_wins'] for d in data['interval_experiments']]

x = np.arange(len(intervals))
ax2.bar(x, video_wins, label='Video', color='#e74c3c', alpha=0.8)
ax2.bar(x, music_wins, bottom=video_wins, label='Music', color='#9b59b6', alpha=0.8)
ax2.bar(x, history_wins, bottom=np.array(video_wins) + np.array(music_wins), 
        label='History', color='#f39c12', alpha=0.8)

ax2.set_xlabel('Junction Interval')
ax2.set_ylabel('Number of Wins')
ax2.set_title('Winner Distribution by Interval')
ax2.set_xticks(x)
ax2.set_xticklabels(intervals)
ax2.legend()

plt.tight_layout()
plt.savefig('../assets/winner_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTotal Wins: Video={total_video}, Music={total_music}, History={total_history}")
print("The agents are well-balanced in their competitiveness.")

## 5. Performance Heatmap

In [ ]:
# Create heatmap data
intervals_list = [1, 2, 5, 10, 30]
timeouts_list = [5, 10, 15, 30]

# Simulated performance matrix (success rate * quality)
performance_matrix = np.array([
    [0.70*65, 0.75*70, 0.78*72, 0.80*75],  # 1s interval
    [0.75*68, 0.80*73, 0.83*76, 0.85*78],  # 2s interval
    [0.85*75, 0.88*78, 0.92*82, 0.95*85],  # 5s interval
    [0.88*78, 0.90*80, 0.93*83, 0.96*86],  # 10s interval
    [0.90*80, 0.92*82, 0.95*85, 0.98*88],  # 30s interval
]) / 100  # Normalize

fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(performance_matrix, cmap='RdYlGn', aspect='auto')

# Add colorbar
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel('Performance Score', rotation=-90, va='bottom')

# Set ticks
ax.set_xticks(np.arange(len(timeouts_list)))
ax.set_yticks(np.arange(len(intervals_list)))
ax.set_xticklabels([f'{t}s' for t in timeouts_list])
ax.set_yticklabels([f'{i}s' for i in intervals_list])

ax.set_xlabel('Agent Timeout')
ax.set_ylabel('Junction Interval')
ax.set_title('Performance Heatmap: Interval vs Timeout')

# Add annotations
for i in range(len(intervals_list)):
    for j in range(len(timeouts_list)):
        text = ax.text(j, i, f'{performance_matrix[i, j]:.2f}',
                       ha='center', va='center', color='black', fontsize=10)

plt.tight_layout()
plt.savefig('../assets/performance_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

# Find optimal
max_idx = np.unravel_index(np.argmax(performance_matrix), performance_matrix.shape)
print(f"\nOptimal configuration: interval={intervals_list[max_idx[0]]}s, timeout={timeouts_list[max_idx[1]]}s")
print(f"Performance score: {performance_matrix[max_idx]:.2f}")

## 6. Statistical Summary

In [ ]:
# Calculate statistics
all_success_rates = [d['success_rate'] for d in data['interval_experiments']]
all_times = [d['total_time'] for d in data['interval_experiments']]

print("="*60)
print("STATISTICAL SUMMARY")
print("="*60)

print("\nProcessing Time Statistics:")
print(f"  Min:      {min(all_times):.1f}s")
print(f"  Max:      {max(all_times):.1f}s")
print(f"  Mean:     {np.mean(all_times):.1f}s")
print(f"  Std Dev:  {np.std(all_times):.1f}s")

print("\nSuccess Rate Statistics:")
print(f"  Min:      {min(all_success_rates)*100:.1f}%")
print(f"  Max:      {max(all_success_rates)*100:.1f}%")
print(f"  Mean:     {np.mean(all_success_rates)*100:.1f}%")

print("\nWinner Distribution:")
total = total_video + total_music + total_history
print(f"  Video:    {total_video} ({total_video/total*100:.1f}%)")
print(f"  Music:    {total_music} ({total_music/total*100:.1f}%)")
print(f"  History:  {total_history} ({total_history/total*100:.1f}%)")

print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
print("\n1. Use junction_interval_seconds=5.0 for balanced performance")
print("2. Use agent_timeout_seconds=30.0 for high reliability")
print("3. All three agents are competitive - no rebalancing needed")

## 7. Export Summary Report

In [ ]:
# Create summary report
summary = {
    'analysis_date': '2024-11-30',
    'optimal_config': {
        'junction_interval_seconds': 5.0,
        'agent_timeout_seconds': 30.0,
        'expected_success_rate': 0.95,
        'expected_processing_time_per_junction': 5.2
    },
    'statistics': {
        'processing_time': {
            'min': min(all_times),
            'max': max(all_times),
            'mean': float(np.mean(all_times)),
            'std': float(np.std(all_times))
        },
        'success_rate': {
            'min': min(all_success_rates),
            'max': max(all_success_rates),
            'mean': float(np.mean(all_success_rates))
        },
        'winner_distribution': {
            'video': total_video,
            'music': total_music,
            'history': total_history
        }
    },
    'visualizations': [
        'assets/interval_analysis.png',
        'assets/timeout_analysis.png',
        'assets/winner_distribution.png',
        'assets/performance_heatmap.png'
    ]
}

with open('../results/analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Summary exported to ../results/analysis_summary.json")
print("\nVisualization images saved to ../assets/")